# 2. Build a labelled dataframe from a cleaned csv file

In [1]:
from pathlib import Path
import sys
import pandas as pd

BASE_PATH = Path.cwd().parent.parent
data_path = BASE_PATH / "data" 
sys.path.append(str(BASE_PATH / "src" / "data"))
import build

/Users/mikeg/miniforge3/envs/osfl2/lib/python3.10/site-packages/opensoundscape/ml/cnn.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Load the processed data - this is a cleaned version of the WildTrax csv data with an additional column for recording_url, latitude and longitude. 

In [2]:
processed_df = pd.read_pickle(data_path / 'interim' / 'cleaned_metadata.pkl')
processed_df.head()

,organization,project,project_id,location,location_id,recording_date_time,recording_id,task_method,task_id,aru_task_status,...,spectrogram_url,clip_url,sensorId,tasks,status,recording_url,latitude,longitude,location_buffer_m,file_type
1623,BU,Alberta Archetypes,1501,P-E0-1-10,308678,2022-06-05 06:51:00,416962,no_restrictions,596169,Transcribed,...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,357,Active,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,52.644040,-115.140510,NaN,flac
1752,BU,Amplitude Quality Testing 2020,293,AM-403-SE2,36043,2017-06-15 04:46:00,92051,no_restrictions,87956,Transcribed,...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,174,Published - Private,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,54.607774,-110.681271,NaN,flac
1758,BU,Amplitude Quality Testing 2020,293,AM-403-SE2,36043,2017-06-15 04:46:00,92051,no_restrictions,87898,Transcribed,...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,174,Published - Private,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,54.607774,-110.681271,NaN,flac
1761,BU,Amplitude Quality Testing 2020,293,AM-403-SE2,36043,2017-06-15 04:46:00,92051,no_restrictions,87840,Transcribed,...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,174,Published - Private,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,54.607774,-110.681271,NaN,flac
1764,BU,Amplitude Quality Testing 2020,293,AM-403-SE2,36043,2017-06-15 04:46:00,92051,no_restrictions,87927,Transcribed,...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,ARU,174,Published - Private,https://wildtrax-aru.s3.us-west-2.amazonaws.co...,54.607774,-110.681271,NaN,flac


If you have an existing test set, you'll want to make sure it doesn't end up in the training data - otherwise the model may be tested on audio it has already seen the labels for.

In [3]:
existing_test_set = pd.read_csv(data_path / 'raw' / "SingleSpecies_all.csv", low_memory=False)

In [4]:
train_and_valid_df, test_df = build.new_labelled_df(processed_df, target_species="OSFL", existing_test_set=existing_test_set, seed=42)

645 not downloaded
downloading 0 clips
skipped 0 previously downloaded files
dropped 69 locations from training set

--------------------------------------------------
train set
recordings per task method = 
 task_method
1SPT               32196
1SPM                9719
no_restrictions     2926
Name: count, dtype: int64
total recordings = 44841

Tags generated from each tagging method:
                 target_present  target_absent
task_method                                   
1SPM                     1402.0         8317.0
1SPT                     1976.0        30220.0
no_restrictions           441.0         2485.0
total present clips =  3819
total absent clips =  41022
total available human labelled tags = 44841

--------------------------------------------------
valid set
recordings per task method = 
 task_method
1SPT               8078
1SPM               2699
no_restrictions     767
Name: count, dtype: int64
total recordings = 11544

Tags generated from each tagging method:
      

# Save the test split somewhere out of the way
Don't look at it until after model training and hyperparameter tuning is complete

In [ ]:
test_df.to_pickle(data_path / 'interim' / 'test' / 'test_set_from_wildtrax.pkl')

# Save the training and validation set in a different folder
This is the data the model will be trained and evaluated on during training. 

In [ ]:
train_and_valid_df.to_pickle(data_path / 'interim' / 'train_and_valid_set' / 'train_and_valid_set_from_wildtrax.pkl')